In [ ]:
from __future__ import absolute_import, division, print_function
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow import keras
import skimage
from skimage import data
from skimage.transform import resize
from PIL import Image
import matplotlib.image as mpimg
import numpy as np
import glob

tf.logging.set_verbosity(tf.logging.INFO)

In [ ]:
directory = '../data/mnist_png/'

In [ ]:
def str_to_vect(label, angle):
    label = int(label)
    angle = int(angle)
    position = (4 - angle)%4
    vector = []
    for i in range(4):
        for j in range(10):
            if i == position and j == label:
                vector.append(1)
            else:
                vector.append(0)
    return vector

In [ ]:
#loaddata
def loaddata(s):
    filelist = glob.glob(directory + s + '/*.png')
    y_temp = []
    x_temp = []
    print (directory + s + '/*.png')
    for fname in filelist:
        short_name = fname.split('/')[-1]
        label, angle, name = short_name.split('_')
        img = mpimg.imread(fname)
        x_temp.append(img)
        vector = str_to_vect(label, angle)
        y_temp.append(vector)

    x = np.asarray(x_temp)
    y = np.asarray(y_temp)
    print (s + ' import done. the shapes of x and y are:', x.shape, y.shape)
    return x, y


In [ ]:
train_x, train_y = loaddata('training')

In [ ]:
test_x, test_y = loaddata('testing')

In [ ]:
plt.figure()
plt.imshow(test_x[5])
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
test_y[5]

In [ ]:
train_x.shape

In [ ]:
def compute_cost(Z, Y):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Z, labels = Y))    
    return cost

In [ ]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
    
    g1_input_layer = tf.image.rot90(input_layer)
    g2_input_layer = tf.image.rot90(input_layer, k=2)
    g3_input_layer = tf.image.rot90(input_layer, k=3)
    # Convolutional Layer #1
    conv1_0 = tf.layers.conv2d(
      inputs=input_layer,
      filters=80,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu,
      reuse = None,
      name = 'conv1')
    
    conv1_1 = tf.layers.conv2d(
      inputs=g1_input_layer,
      filters=80,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu,
      reuse = True,
      name = 'conv1')    

    conv1_2 = tf.layers.conv2d(
      inputs=g2_input_layer,
      filters=80,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu,
      reuse = True,
      name = 'conv1')  

    conv1_3 = tf.layers.conv2d(
      inputs=g3_input_layer,
      filters=80,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu,
      reuse = True,
      name = 'conv1') 
    
    conv1    = tf.concat([conv1_0, conv1_1, conv1_2, conv1_3], axis = 3)
    g1_conv1 = tf.concat([conv1_1, conv1_2, conv1_3, conv1_0], axis = 3)
    g2_conv1 = tf.concat([conv1_2, conv1_3, conv1_0, conv1_1], axis = 3)
    g3_conv1 = tf.concat([conv1_3, conv1_0, conv1_1, conv1_2], axis = 3)
    
    # Convolutional Layer #2 and Pooling Layer #2
    conv2_0 = tf.layers.conv2d(
      inputs=conv1,
      filters=80,
      kernel_size=[3, 3],
      padding="same",
      reuse = None,
      activation=tf.nn.relu,
      name = 'conv2')
    
    conv2_1 = tf.layers.conv2d(
      inputs=g1_conv1,
      filters=80,
      kernel_size=[3, 3],
      padding="same",
      reuse = True,
      activation=tf.nn.relu,
      name = 'conv2')

    conv2_2 = tf.layers.conv2d(
      inputs=g2_conv1,
      filters=80,
      kernel_size=[3, 3],
      padding="same",
      reuse = True,
      activation=tf.nn.relu,
      name = 'conv2')

    conv2_3 = tf.layers.conv2d(
      inputs=g3_conv1,
      filters=80,
      kernel_size=[3, 3],
      padding="same",
      reuse = True,
      activation=tf.nn.relu,
      name = 'conv2')
    
    conv2    = tf.concat([conv2_0, conv2_1, conv2_2, conv2_3], axis = 3)
    g1_conv2 = tf.concat([conv2_1, conv2_2, conv2_3, conv2_0], axis = 3)
    g2_conv2 = tf.concat([conv2_2, conv2_3, conv2_0, conv2_1], axis = 3)
    g3_conv2 = tf.concat([conv2_3, conv2_0, conv2_1, conv2_2], axis = 3)

    conv3_0 = tf.layers.conv2d(
      inputs=conv2,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      reuse = None,
      activation=tf.nn.relu,
      name = 'conv3')

    conv3_1 = tf.layers.conv2d(
      inputs=g1_conv2,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      reuse = True,
      activation=tf.nn.relu,
      name = 'conv3')

    conv3_2 = tf.layers.conv2d(
      inputs=g2_conv2,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      reuse = True,
      activation=tf.nn.relu,
      name = 'conv3')

    conv3_3 = tf.layers.conv2d(
      inputs=g3_conv2,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      reuse = True,
      activation=tf.nn.relu,
      name = 'conv3')
    
    conv3    = tf.concat([conv3_0, conv3_1, conv3_2, conv3_3], axis = 3)
    g1_conv3 = tf.concat([conv3_1, conv3_2, conv3_3, conv3_0], axis = 3)
    g2_conv3 = tf.concat([conv3_2, conv3_3, conv3_0, conv3_1], axis = 3)
    g3_conv3 = tf.concat([conv3_3, conv3_0, conv3_1, conv3_2], axis = 3)
    
    pool4_0 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[4, 4], strides=4)
    pool4_1 = tf.layers.max_pooling2d(inputs=g1_conv3, pool_size=[4, 4], strides=4)
    pool4_2 = tf.layers.max_pooling2d(inputs=g2_conv3, pool_size=[4, 4], strides=4)
    pool4_3 = tf.layers.max_pooling2d(inputs=g3_conv3, pool_size=[4, 4], strides=4)
    
    pool4_0_flat = tf.reshape(pool4_0, [-1, 7 * 7 * 128])
    pool4_1_flat = tf.reshape(pool4_1, [-1, 7 * 7 * 128])
    pool4_2_flat = tf.reshape(pool4_2, [-1, 7 * 7 * 128])
    pool4_3_flat = tf.reshape(pool4_3, [-1, 7 * 7 * 128])
        
    dense_0 = tf.layers.dense(inputs=pool4_0_flat, 
                              units=1024, 
                              activation=tf.nn.relu,
                              reuse = None,
                              name = 'dense')

    dense_1 = tf.layers.dense(inputs=pool4_1_flat, 
                              units=1024, 
                              activation=tf.nn.relu,
                              reuse = True,
                              name = 'dense')

    dense_2 = tf.layers.dense(inputs=pool4_2_flat, 
                              units=1024, 
                              activation=tf.nn.relu,
                              reuse = True,
                              name = 'dense')
    
    dense_3 = tf.layers.dense(inputs=pool4_3_flat, 
                              units=1024, 
                              activation=tf.nn.relu,
                              reuse = True,
                              name = 'dense')

    # Logits Layer
    logits_0 = tf.layers.dense(inputs=dense_0,
                               units=10,
                               reuse = None,
                               name = 'logi'
                              )

    logits_1 = tf.layers.dense(inputs=dense_1, 
                               units=10,
                               reuse = True,
                               name = 'logi'
                              )

    
    logits_2 = tf.layers.dense(inputs=dense_2, 
                               units=10,
                               reuse = True,
                               name = 'logi'
                              )
    
    logits_3 = tf.layers.dense(inputs=dense_3, 
                               units=10,
                               reuse = True,
                               name = 'logi'
                              )
    
    final = tf.concat([logits_0, logits_1, logits_2, logits_3], axis = 1)
    
    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=final, axis= 1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(final, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = compute_cost(final, labels)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=tf.argmax(input=labels, axis= -1), predictions=predictions["classes"])
    }
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [ ]:
train_data = train_x / 255.0
eval_data = test_x / 255.0

train_labels = train_y.astype(np.int32)  # not required
eval_labels = test_y.astype(np.int32)  # not required

In [ ]:
train_data.shape, train_labels.shape

In [ ]:
eval_data.shape, eval_labels.shape

In [ ]:
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn)

In [ ]:
tensors_to_log = {"probabilities": "softmax_tensor"}

logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

In [ ]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=200,
    num_epochs=None,
    shuffle=True)

# train one step and display the probabilties
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=1,
    hooks=[logging_hook])

In [ ]:
mnist_classifier.train(input_fn=train_input_fn, steps=2000)

In [ ]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

# Now we test equivariancy on some examples

In [ ]:
samples_x, samples_y = loaddata('samples')
samples_x = samples_x/np.float32(255)

In [ ]:
samples_x.shape

In [ ]:
plt.figure()
plt.imshow(samples_x[0])
plt.colorbar()
plt.grid(False)

plt.figure()
plt.imshow(samples_x[1])
plt.colorbar()
plt.grid(False)

plt.figure()
plt.imshow(samples_x[2])
plt.colorbar()
plt.grid(False)

plt.figure()
plt.imshow(samples_x[3])
plt.colorbar()
plt.grid(False)

plt.show()

In [ ]:
pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": samples_x},
    shuffle=False)

In [ ]:
predictions = mnist_classifier.predict(input_fn = pred_input_fn)

In [ ]:
def decode(c):
    return c%10, (4 -(c-c%10)/10)%4 * 90

In [ ]:
for prediction in predictions:
    print (decode(prediction['classes']), '\n', prediction['probabilities'])